In [1]:
import numpy as np
import pandas as pd
import os
import glob
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import  RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from google.colab import drive
from google.colab import files
from scipy.stats import ttest_ind
import seaborn as sns
from sklearn.preprocessing import LabelEncoder


In [2]:


drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/data /content/local_data


activity_labels=['idle', 'running', "stairs", "walking"]
files={label:[] for label in activity_labels}

Mounted at /content/drive


In [3]:
data='/content/local_data'



In [4]:
for root,dirs,filles in os.walk(data):


    for file in filles:
        for label in activity_labels :
            if label in file and  file.endswith("csv"):
                path = os.path.join(root, file)
                try:
                    read = pd.read_csv(path)
                    read['activity'] = label
                    files[label].append(read)

                except Exception as e:
                    print(f"Błąd podczas wczytywania pliku: {path}")
                    print(f"Szczegóły błędu: {e}")


In [5]:
if any(files.values()):
    data_fatms=[]
    for label in activity_labels:
        if files[label]:
            data_fatms.append(pd.concat(files[label], ignore_index=True))
    data=pd.concat(data_fatms, ignore_index=True)
    data=data.dropna()
    x = data[["accelerometer_X", "accelerometer_Y", "accelerometer_Z"]].values
    y = data["activity"].values

le = LabelEncoder()
y = le.fit_transform(data["activity"])


scalar = StandardScaler()
data_scaled = scalar.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(data_scaled,y, test_size=0.4, random_state=40)
x_scalar=scalar.fit_transform(X_train)
x_test_scalar=scalar.fit_transform(X_test)

pca=PCA(n_components=3)
x_trein_pca=pca.fit_transform(x_scalar)
x_test_pca=pca.transform(x_test_scalar)

laso=Lasso(alpha=0.1)
laso.fit(x_scalar,y_train)
model=SelectFromModel(laso)
x_trein_lasso=model.transform(x_scalar)
x_test_lasso=model.transform(x_test_scalar)


model_rf =  RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_rf.fit(x_scalar, y_train)

model_svc = SVC(kernel="rbf", C=1, gamma="auto")
model_svc.fit(X_train, y_train)

model_rf =  RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_rf.fit(x_scalar, y_train)

model_svc = SVC(kernel="rbf", C=1, gamma="auto")
model_svc.fit(x_scalar, y_train)

SVC(C=1, gamma='auto')

In [6]:
def ewaluacja(model, X_train, X_test, y_train, y_test ):
  model.fit(X_train, y_train)
  y_pred=model.predict(X_test)
  y_prob=model.predict_proba(X_test)
  if y_prob.ndim == 1:
    y_prob = np.column_stack((1 - y_prob, y_prob))
  accuaracy=accuracy_score(y_test, y_pred)
  precision=precision_score(y_test, y_pred, average='macro')
  recall=recall_score(y_test, y_pred, average='macro')
  f1=f1_score(y_test, y_pred, average='macro')
  roc_auc=roc_auc_score(y_test, y_prob,multi_class='ovo')
  return accuaracy, precision, recall, f1, roc_auc

In [7]:
rezult=[]
ferniture_set={'All features':(x_scalar, x_test_scalar),'PCA features':(x_trein_pca, x_test_pca),'Lasso features':(x_trein_lasso,x_test_lasso)}
svm=SVC(kernel='rbf', C=1, gamma='auto',probability=True)

for name, (X_tr, X_te) in ferniture_set.items():

  rezult.append(("SVM", name, *ewaluacja(svm,X_tr,X_te,y_train,y_test)))
  rezult.append(("Random forest", name, *ewaluacja(model_rf,X_tr, X_te, y_train,y_test)))
result_df = pd.DataFrame(rezult, columns=['Model', 'Feature Set', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC'])
print(result_df)

           Model     Feature Set  Accuracy  Precision    Recall  F1 Score  \
0            SVM    All features  0.894176   0.923013  0.699210  0.686340   
1  Random forest    All features  0.924391   0.946959  0.731973  0.736092   
2            SVM    PCA features  0.894176   0.923013  0.699210  0.686340   
3  Random forest    PCA features  0.920677   0.944637  0.719889  0.716538   
4            SVM  Lasso features  0.838543   0.885693  0.673110  0.650107   
5  Random forest  Lasso features  0.861782   0.883068  0.687193  0.678785   

    ROC AUC  
0  0.915859  
1  0.966433  
2  0.915984  
3  0.966160  
4  0.885219  
5  0.940601  


In [12]:
model_rf =  RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model_rf.fit(x_scalar, y_train)
predic_svm=model_svc.predict(X_test)

perfect_svm=accuracy_score(y_test, predic_svm)
predic_rf=model_rf.predict(X_test)
perfect_rf=accuracy_score(y_test, predic_rf)

In [13]:
data['accelerometer_X_mean']= data['accelerometer_X'].rolling(window=10).mean()
data['accelerometer_X_std']= data['accelerometer_X'].rolling(window=10).std()
data['accelerometer_X_min']= data['accelerometer_X'].rolling(window=10).min()
data['accelerometer_X_max']= data['accelerometer_X'].rolling(window=10).max()
data['accelerometer_X_median']= data['accelerometer_X'].rolling(window=10).median()

data['accelerometer_Y_mean']= data['accelerometer_Y'].rolling(window=10).mean()
data['accelerometer_Y_std']= data['accelerometer_Y'].rolling(window=10).std()
data['accelerometer_Y_min']=data['accelerometer_Y'].rolling(window=10).min()
data['accelerometer_Y_max']=data['accelerometer_Y'].rolling(window=10).max()
data['accelerometer_Y_median']= data['accelerometer_Y'].rolling(window=10).median()

data['accelerometer_Z_mean']= data['accelerometer_Z'].rolling(window=10).mean()
data['accelerometer_Z_std']= data['accelerometer_Z'].rolling(window=10).std()
data['accelerometer_Z_min']= data['accelerometer_Z'].rolling(window=10).min()
data['accelerometer_Z_max']= data['accelerometer_Z'].rolling(window=10).max()
data['accelerometer_Z_median']= data['accelerometer_Z'].rolling(window=10).median()

data= data.dropna()

fernitur=['accelerometer_X_mean','accelerometer_X_std','accelerometer_X_min','accelerometer_X_max','accelerometer_X_median',
    'accelerometer_Y_mean','accelerometer_Y_std','accelerometer_Y_min','accelerometer_Y_max','accelerometer_Y_median',
    'accelerometer_Z_mean','accelerometer_Z_std','accelerometer_Z_min','accelerometer_Z_max','accelerometer_Y_median']
x=data[fernitur].values

data_scalar=scalar.fit_transform(x)
if len(data_scalar)!= len(y):
    o=(len(y))
    p=(len(data_scalar))


    q=p-o
    if  o>p:
      y=np.delete(y,range(-q))
    if p>o:
      data_scalar=np.delete(data_scalar,range(-q))
else:
  y=y
  data_scalar=data_scalar



In [14]:

x_train, x_test, y_train, y_test = train_test_split(data_scalar, y, test_size=0.4, random_state=40)
model={'SVM': model_svc,"Losowy":model_rf}
score={}
for names_model, modeles in model.items():

        score[names_model]=cross_val_score(modeles,x_train,y_train,cv=5).mean()

        #score[names_model]=score

for names_model, scor in score.items():
        print(f'{names_model}:{scor}')


SVM:0.9848336342532887
Losowy:0.9930100593242198


In [ ]:
from google.colab import drive
drive.mount('/content/drive')